In [1]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import dotenv
import time
dotenv.load_dotenv()
import os

llm = ChatOpenAI(model="gpt-4o", api_key=os.getenv('OPENAI_API_KEY'))
embeddings = OpenAIEmbeddings()

In [2]:
# Load my LLM 

import sys
import os
sys.path.append('../..')


from backend.llms.llm_rag import LLMRag
from backend.llms.llm_gemini import LLMGemini

backend_llm = LLMGemini()
llmrag = LLMRag(backend=backend_llm)
response = llmrag.process_request('Ce e un subquery  ?')

config {'chunker_name': 'chunks', 'similarity_threshold': 0.2, 'top_k': 10, 'no_internal_source_prompt': 'Your only answer should be exactly like this: Nu am gasit informatii in curs despre intrebarea ta', 'internal_source_rules': 'Esti un asistent care raspunde in limba Romana la intrebarea din  "ORIGINAL USER PROMPT" folosind context din "CONTEXT FROM INTERNAL SOURCES". Raspunde folosind numai informatii din context. Raspunsul tau trebuie sa fie complet si sa contina cat mai mult din contextul intrebarii. Vei raspunde \'Folosind informatiile din curs, \' si apoi vei raspunde'} 
 -------------------------------
candidates {
  content {
    role: "model"
    parts {
      text: "Folosind informatiile din curs, o subinterogare (subquery) poate fi plasata în clauzele WHERE, HAVING  sau FROM. Subinterogarea se execută prima dată, iar rezultatul este folosit pentru obţinerea rezultatului de către interogarea principală (outer query). \n"
    }
  }
  finish_reason: STOP
  safety_ratings {
 

In [3]:
import json
intrebari_dict =json.load(open('intrebari_sql.json','rt'))

In [4]:
intrebari_dict

{'q1': {'query': 'Ce este SQL?',
  'expected_response': 'SQL este un limbaj de interogare a bazelor de date.'},
 'q2': {'query': 'Care este sintaxa de bază a unei interogări SELECT?',
  'expected_response': 'SELECT [DISTINCT] lista_de_expresii FROM nume_tabela WHERE conditie_linie ORDER BY criterii_sortare_rezultat;'},
 'q3': {'query': 'Ce face clauza WHERE într-o interogare SQL?',
  'expected_response': 'Clauza WHERE restricționează rândurile returnate la cele care îndeplinesc condiția specificată.'},
 'q4': {'query': 'Care este efectul utilizării DISTINCT într-o interogare?',
  'expected_response': 'DISTINCT elimină rândurile duplicat din rezultatul interogării.'},
 'q5': {'query': 'Ce face clauza ORDER BY?',
  'expected_response': 'Clauza ORDER BY sortează rezultatele în funcție de coloanele specificate.'},
 'q6': {'query': 'Ce este o funcție în SQL?',
  'expected_response': 'Funcțiile sunt utilizate pentru a realiza calcule asupra datelor, a modifica date și a manipula grupuri de î

In [5]:
sample_queries = [intrebari_dict[x]['query'] for x in intrebari_dict]
expected_responses = [intrebari_dict[x]['expected_response'] for x in intrebari_dict]

In [ ]:
dataset = []
i=0
for query,reference in zip(sample_queries,expected_responses):
    print(f'question{i}')
    i+=1
    full_response = llmrag.process_request(query)
    time.sleep(10)
    relevant_docs = full_response.context
    response = full_response.text
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":relevant_docs,
            "response":response,
            "reference":reference
        }
    )

candidates {
  content {
    role: "model"
    parts {
      text: "Folosind informatiile din curs, SQL (Limbajul structurat de interogare) este un limbaj utilizat pentru a interoga baze de date. Un exemplu de cerere SQL este `SELECT empno, UPPER(ename), job, deptno FROM EMP WHERE INITCAP(ename) = \'Smith\';`, care selectează anumite coloane din tabela `EMP` unde numele, convertit la majuscule la început, este \'Smith\'.\n"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0705596507
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.151027903
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.0665691793
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0637151301
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.0838904828
    severity: H

In [9]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [10]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper


evaluator_llm = LangchainLLMWrapper(llm)
from ragas.metrics import ContextPrecision,  LLMContextRecall, AnswerRelevancy,  Faithfulness,  BleuScore, RougeScore, FactualCorrectness
metrics = [
        ContextPrecision(),
        LLMContextRecall(),
        AnswerRelevancy(),
        Faithfulness(),
        BleuScore(),
        RougeScore(),
        FactualCorrectness()]

result = evaluate(
    dataset=evaluation_dataset,
    metrics=metrics,
    llm=evaluator_llm)
result

ImportError: sacrebleu is required for bleu score. Please install it using `pip install sacrebleu`

In [12]:
import os
os.environ["RAGAS_APP_TOKEN"] = "apt.4418-2f131daa8aa7-086e-8dd7-0123d22b-08626"

In [ ]:
result.upload()